# **AutoGen Quick Start**
* https://microsoft.github.io/autogen/stable//user-guide/agentchat-user-guide/tutorial/models.html#azure-openai  

먼저 같은 폴더에 ".env" 파일을 생성하고 아래 정보를 기록.  

AZURE_OPENAI_API_KEY = "azure openai api key"  
LLM_API_VERSION = "llm api version"  
LLM_DEPLOYMENT = "llm deployment"  
LLM_MODEL_VERSION = "llm model version"  
LLM_ENDPOINT = "llm endpoint"  
LLM_NAME = "llm name"  
LLM_TEMP = 0.1  

In [22]:
## pip install -U "autogen-agentchat" "autogen-ext[openai,azure]"

In [8]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings

import os

llm = AzureChatOpenAI(
        azure_deployment = os.environ["LLM_DEPLOYMENT"],
        azure_endpoint = os.environ["LLM_ENDPOINT"],
        openai_api_version = os.environ["LLM_API_VERSION"],
        openai_api_key = os.environ["AZURE_OPENAI_API_KEY"],
        model_version= os.environ["LLM_MODEL_VERSION"],
        temperature = 0.1,
)

#Azure text embedding
########################################################
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large-1",
    # dimensions: Optional[int] = None, # Can specify dimensions with new text-embedding-3 models
    azure_endpoint="https://dev.dxengws.apim.lgedx.biz/shared-embedding", #If not provided, will read env variable AZURE_OPENAI_ENDPOINT
    # api_key=... # Can provide an API key directly. If missing read env variable AZURE_OPENAI_API_KEY
    # openai_api_version=..., # If not provided, will read env variable AZURE_OPENAI_API_VERSION
)
########################################################
print("ALL OK")

ALL OK


In [36]:
from autogen_core.models import UserMessage
from autogen_ext.auth.azure import AzureTokenProvider

# from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
## 사용하려는 모델을 미리 설치해야함
## pip install -U "autogen-agentchat" "autogen-ext[openai,azure]"

from azure.identity import DefaultAzureCredential

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console

# # Create the token provider
# token_provider = AzureTokenProvider(
#     DefaultAzureCredential(),
#     "https://cognitiveservices.azure.com/.default",
# )

model_client  = AzureOpenAIChatCompletionClient(
    azure_deployment = os.environ["LLM_DEPLOYMENT"],
    model = os.environ["LLM_NAME"],
    api_version= os.environ["LLM_API_VERSION"],
    azure_endpoint= os.environ["LLM_ENDPOINT"],
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key = os.environ["AZURE_OPENAI_API_KEY"] # For key-based authentication.
)

result = await model_client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)
await az_model_client.close()

finish_reason='stop' content='The capital of France is **Paris**.' usage=RequestUsage(prompt_tokens=15, completion_tokens=10) cached=False logprobs=None thought=None


/tmp/ipykernel_3378011/2276180793.py:26: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  result = await model_client.create([UserMessage(content="What is the capital of France?", source="user")])


## 에이젼트가 사용할 툴 정의

In [51]:
model_client  = AzureOpenAIChatCompletionClient(
    azure_deployment = os.environ["LLM_DEPLOYMENT"],
    model = os.environ["LLM_NAME"],
    api_version= os.environ["LLM_API_VERSION"],
    azure_endpoint= os.environ["LLM_ENDPOINT"],
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key = os.environ["AZURE_OPENAI_API_KEY"] # For key-based authentication.
)

In [37]:
# Define a simple function tool that the agent can use.
# For this example, we use a fake weather tool for demonstration purposes.
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."

In [54]:
async def get_stock_price(company_name: str) -> str:
    """Get the stock price of given company."""
    return f"The current stock price of {company_name} is 12345678900 won."

### Agent 생성


### reflect_on_tool_use 파라미터
    
reflect_on_tool_use는 AssistantAgent의 생성자에서 설정할 수 있는 불리언(Boolean) 파라미터입니다. 이 파라미터는 에이전트가 도구를 사용한 후 그 결과에 대해 추가적인 추론을 수행할지 여부를 결정합니다.

* True인 경우: 에이전트는 도구 호출 및 결과에 대해 추가적인 모델 추론을 수행하여 최종 응답을 생성합니다. 이는 도구 사용 결과를 요약하거나, 더 자세한 분석을 제공하는 데 유용합니다.
* False인 경우: 에이전트는 도구 호출 결과를 직접 응답으로 반환합니다. 이 경우 도구에서 반환된 결과가 자연어 형태인지, 다른 에이전트가 특정 스키마를 기대하는지 주의해야 합니다.

기본적으로 output_content_type이 설정되면 reflect_on_tool_use는 True로 설정되고, 그렇지 않으면 False로 설정됩니다.

In [55]:
# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather, get_stock_price],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True, # 도구 사용 결과에 대해 reflection을 수행하도록 설정
    model_client_stream=True,  # Enable streaming tokens from the model client.
)

In [56]:
# Run the agent and stream the messages to the console.
async def main() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))

    await Console(agent.run_stream(task="What is the current stock price of AI odyssey company?"))
    # Close the connection to the model client.    
    await model_client.close()


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

---------- TextMessage (user) ----------
What is the weather in New York?
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='call_rW6EOmmH2yZRGmmLraGjEfTT', arguments='{"city":"New York"}', name='get_weather')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', name='get_weather', call_id='call_rW6EOmmH2yZRGmmLraGjEfTT', is_error=False)]
---------- ModelClientStreamingChunkEvent (weather_agent) ----------
The weather in New York is currently 73 degrees and sunny.
---------- TextMessage (user) ----------
What is the current stock price of AI odyssey company?
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='call_4a1qzft8Aosib9H9tOeVqXyF', arguments='{"company_name":"AI Odyssey"}', name='get_stock_price')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='The current stock price of AI Odyssey is 12

### Test #1. Tool 함수의 이름을 보고 호출하는 것인가?  
이름을 엉뚱하게 바꾸어 보자.  get_weather --> do_addition_of_given_number

In [57]:
model_client  = AzureOpenAIChatCompletionClient(
    azure_deployment = os.environ["LLM_DEPLOYMENT"],
    model = os.environ["LLM_NAME"],
    api_version= os.environ["LLM_API_VERSION"],
    azure_endpoint= os.environ["LLM_ENDPOINT"],
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key = os.environ["AZURE_OPENAI_API_KEY"] # For key-based authentication.
)

In [58]:
# Define a simple function tool that the agent can use.
# For this example, we use a fake weather tool for demonstration purposes.
# async def get_weather(city: str) -> str:
#     """Get the weather for a given city."""
#     return f"The weather in {city} is 73 degrees and Sunny."

async def do_addition_of_given_number(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."

In [59]:
async def get_stock_price(company_name: str) -> str:
    """Get the stock price of given company."""
    return f"The current stock price of {company_name} is 12345678900 won."

In [60]:
# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[do_addition_of_given_number, get_stock_price],  ##!!!! 이름 변경
    system_message="You are a helpful assistant.If you can't answer the question, just say I can't answer your question.",
    reflect_on_tool_use=True, # 도구 사용 결과에 대해 reflection을 수행하도록 설정
    model_client_stream=True,  # Enable streaming tokens from the model client.
)

In [61]:
# Run the agent and stream the messages to the console.
async def main() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))

    await Console(agent.run_stream(task="What is the current stock price of AI odyssey company?"))
    # Close the connection to the model client.    
    await model_client.close()


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

### Tool 호출 잘함

---------- TextMessage (user) ----------
What is the weather in New York?
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='call_VP47CcGzuC3ivojEBNwDLoD3', arguments='{"city":"New York"}', name='do_addition_of_given_number')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', name='do_addition_of_given_number', call_id='call_VP47CcGzuC3ivojEBNwDLoD3', is_error=False)]
---------- ModelClientStreamingChunkEvent (weather_agent) ----------
The weather in New York is 73 degrees and sunny.
---------- TextMessage (user) ----------
What is the current stock price of AI odyssey company?
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='call_zNBj03z3PmOACdH5DivqUI8w', arguments='{"company_name":"AI odyssey"}', name='get_stock_price')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='The current stock pri

### Test #2. Tool 함수 내부의 doc string 보고 호출하는 것인가?  
doc string 내용을 엉뚱하게 바꾸어 보자.  

In [74]:
model_client  = AzureOpenAIChatCompletionClient(
    azure_deployment = os.environ["LLM_DEPLOYMENT"],
    model = os.environ["LLM_NAME"],
    api_version= os.environ["LLM_API_VERSION"],
    azure_endpoint= os.environ["LLM_ENDPOINT"],
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key = os.environ["AZURE_OPENAI_API_KEY"] # For key-based authentication.
)

In [75]:
# Define a simple function tool that the agent can use.
# For this example, we use a fake weather tool for demonstration purposes.
# async def get_weather(city: str) -> str:
#     """Get the weather for a given city."""
#     return f"The weather in {city} is 73 degrees and Sunny."

async def get_weather(city: str) -> str:
    """Get the poplulations for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."

In [76]:
async def get_stock_price(company_name: str) -> str:
    """Get the doors of given company."""
    return f"The current stock price of {company_name} is 12345678900 won."

In [77]:
# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather, get_stock_price],  ##!!!! 이름 변경
    system_message="You are a helpful assistant.If you can't answer the question, just say I can't answer your question.",
    reflect_on_tool_use=True, # 도구 사용 결과에 대해 reflection을 수행하도록 설정
    model_client_stream=True,  # Enable streaming tokens from the model client.
)

In [78]:
# Run the agent and stream the messages to the console.
async def main() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))

    await Console(agent.run_stream(task="What is the current stock price of AI odyssey company?"))
    # Close the connection to the model client.    
    await model_client.close()


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

### Tool 호출 잘함

---------- TextMessage (user) ----------
What is the weather in New York?
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='call_9BVDpy16WtHB4Hc5fLvwYN3x', arguments='{"city":"New York"}', name='get_weather')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', name='get_weather', call_id='call_9BVDpy16WtHB4Hc5fLvwYN3x', is_error=False)]
---------- ModelClientStreamingChunkEvent (weather_agent) ----------
The weather in New York is 73 degrees and sunny.
---------- TextMessage (user) ----------
What is the current stock price of AI odyssey company?
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='call_ReHPXgmaVNjcp780YnesAfEC', arguments='{"company_name":"AI odyssey"}', name='get_stock_price')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='The current stock price of AI odyssey is 12345678900 

---
**AutoGen Tool Calling 판단 기준**  
    
autogen 프레임워크에서 tool을 사용하기 위해 LLM은 다음 정보들을 활용합니다:  

* 함수 이름:  tool의 이름은 LLM이 어떤 tool을 호출할지 결정하는 데 중요한 역할을 합니다.  
Description (설명): tool의 목적과 사용법에 대한 설명을 제공하여 LLM이 tool을 이해하고 적절하게 호출하도록 돕습니다. 이 설명은 tool을 agent에 등록할 때 제공됩니다.  
* Type Hints (타입 힌트): 함수의 인자와 반환 값에 대한 타입 힌트는 LLM이 tool의 사용법을 이해하는 데 도움을 줍니다.  
* Tool Schema:  함수 시그니처와 타입 힌트로부터 자동 생성되는 tool 스키마는 LLM이 tool을 어떻게 사용해야 하는지에 대한 구조화된 정보를 제공합니다.  
* Docstring: Ollama LLM과 같은 특정 LLM은 tool을 인지하기 위해 Python과 유사한 구문으로 작성된 docstring이 필요할 수 있습니다.  